**Моделирование распределения длин N1 зоны антител человека**

In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
import os
from scipy.optimize import minimize
from scipy.stats import chisquare
os.chdir("/Users/irina/Desktop/IG/small_phenotypes")

Моделирование проводится отдельно для различных фенотипов (источников клеток) и только для productive последовательностей. Исходный датасет был разделен на фенотипы, взятые из названий статей. Создается словарь *phen_hist_dict*, ключи для которого - название фенотипа, а значения -  array, содержащий частоты длин N1 зоны (например, если нулевой элемент массива = 20, то N1 зона длины 1 встретилась в данных 20 раз). Те последовательности, где N1 зона была равна нулю, были исключены из анализа. Еще можно удалить клонов, которые были найдены с помощью тула Partis для того, чтобы выборка была случайной (а можно не удалять, тогда нужно пропустить следующий блок).

In [2]:
list_ids = [] #list of ids from Partis output, the first id from each clonal family
for n in range(1,162):
    try:
        aa= pd.read_csv("./1-162/" + str(n) + '-cluster-annotations.csv')
        for _,row in aa.iterrows():
            list_ids.append(row["unique_ids"].split(":")[0])
    except:
        continue

In [3]:
def cut_num(paper_name):
    x = re.search('[0-9]+\)\s', paper_name)    
    start_pos = x.end()
    return paper_name[start_pos:]

def make_ID_dict(fname):
    ID_dict = {}
    counter = 0
    with open(fname,'r') as f:
        for line in f:
            counter += 1
            if counter%3 == 1:
                ID = line.strip()
            elif counter%3 == 2:
                paper = line.strip()
                ID_dict[ID] = paper
    return ID_dict

handle = pd.ExcelFile('table_phenotypes.xlsx')
data = handle.parse(sheetname="Sheet1",skiprows=1)
phenotypes = set(data.columns)
phen_exclude = set(['Unnamed: 23', '//T helper cells', 'Unnamed: 31', '//SHM and Absence of AID','//Primary cold agglutinin disease','//Anti-platelets Ab','//Human Ig combinatorial library from genomic V segments and synthetic CDR3 fragments','//In vitro assembly and Sterile DJH rearrangements','//Women with myasthenia: evidence for immunization by fetal antigen','//Xeroderma pigmentosum','//IgD-only B cells','//T helper cells'])
phenotypes.difference_update(phen_exclude)

paper_dict = {cut_num(paper):phen for phen in phenotypes for paper in data[phen] if (type(paper)==str and ')' in paper)}
ID_dict = make_ID_dict('id_title-productive_seqs.txt')
ID_phen_dict = {ID:paper_dict[paper] for ID,paper in ID_dict.items() if paper in paper_dict}

len_data = pd.read_csv('Nt_seq.csv') #V-quest output
len_data = len_data.loc[len_data["lens_n1"]>0]
phen_hist_dict = {phenotype:np.zeros((1000)) for phenotype in phenotypes}
for _,row in len_data.iterrows():
    current_ID = row['Sequence ID']
    #if current_ID in list_ids: #if you don't need filter sequences from one clonal family, just delete or comment this line
    if current_ID in ID_phen_dict:
        phen_hist_dict[ID_phen_dict[current_ID]][int(row['lens_n1']) - 1] += 1
sortednames=sorted(phen_hist_dict.keys(), key=lambda x:x.lower()) 
phen_hist_dict = {sor:phen_hist_dict[sor] for sor in sortednames}

Функция *pnpe* содержит модель распределения длин N1 зоны. Вероятность нулевой длины = 0. Функция *likelihood* нужна для поиска оптимальных параметров модели.

In [4]:
def pnpe(n, pp, pn, pe):
    if n > 0:
        prob = ((pe**2*pn*pp**2*((n-1)*(pe*(1-pp))**(n-2)))/((pn-pp)*(pe+pp-pe*pp)**(n+2))) + ((pe*pn*pp**2*(pe*(1-pn))**n)/((pn-pp)**2*(pe-pe*pn)*(pe+pn-pe*pn)**(n+1))) - ((pe*pn*pp**2 * (pe*(1-pp))**n * (pe-2*pn + 3*pp + 2* pe*pn - 3 *pe*pp))/((pn - pp)**2*(pe-pe*pp)*(pe + pp - pe*pp)**(n+2)))
        return prob
    elif n == 0:
        prob = 0
        return prob
    else:
        prob = ((pe**2*pn*pp**2*((1/(pe**2*(pp-1)**2))+(((n-1)*((-pe*(pp-1))/(pe+pp-pe*pp))**(n-2))/((pe+pp-pe*pp)**2))))/((pn-pp)*(pe-pp-pe*pp)**2)) - ((pn*pp**2*(pe-1)**3)/((pe+pn - pe*pn)*(pe+pp - pe*pp)**2)) +((pe*pn*pp**2*(((-pe*(pn-1))/(pe+pn-pe*pn))**n-1))/((pn-pp)**2*(pe-pe*pn)*(pe+pn-pe*pn))) - ((pe*pn*pp**2*(((-pe*(pp-1))/(pe+pp-pe*pp))**n-1)*(pe-2*pn+3*pp+2*pe*pn - 3*pe*pp))/((pn - pp)**2*(pe-pe*pp)*(pe+pp-pe*pp)**2))
        return prob

def likelihood(params):
    pp, pn, pe = params[0], params[1], params[2]
    return -np.sum([(y_exp_norm[x0-1])*np.log(pnpe(x0, pp, pn, pe)) for x0 in x_exp])

Функция *optimize_initial_guess* подбирает начальные приближенные параметры для функции максимального правдоподобия. Поскольку для этого надо перебрать 1 тыс комбинаций, лучше не запускать ее каждый раз. В прошлый раз оптимальный initial guess был 0.91, 0.81, 0.01.

In [20]:
def optimize_initial_guess():
    aa = np.meshgrid(np.arange(0.01, 1, 0.1), np.arange(0.01, 1, 0.1), np.arange(0.01, 1, 0.1)) #чтобы перебрать 1 млн комбинаций, измените шаг на 0.01
    coord_list = [entry.ravel() for entry in aa]
    points = np.vstack(coord_list).T
    res_dict = {}
    n = 0
    for point in points:
        result = minimize(likelihood, x0=point, bounds=((0.2, 0.999), (0.01, 0.999), (0.01, 0.999))) #minimization of ML function
        res_dict[result.fun] = point
        n+=1
    key_list = [x for x in res_dict.keys() if str(x) != 'nan']
    return res_dict[min(key_list)]


In [6]:
parametrs = []
all_data = []
#find the parameters for each phenotype
for phen in phen_hist_dict:
    if sum(phen_hist_dict[phen]) >= 80: #only for more than 80 samples
        x_exp = list(np.arange(1, 51))
        y_exp = list(phen_hist_dict[phen][:50])
        y_exp_norm = [y / sum(y_exp) for y in y_exp]
        #initial_guess = optimize_initial_guess() #Если не хочется подбирать изначальные параметры, надо закомментировать эту строку и активировать следующую
        initial_guess = np.array([0.91, 0.81, 0.01])
        result = minimize(likelihood, x0=initial_guess, bounds=((0.2, 0.999), (0.01, 0.999), (0.001, 0.999))) #minimization of ML function
        coeff = result.x
        y = [pnpe(z, coeff[0], coeff[1], coeff[2]) for z in x_exp]
        parametrs.append([coeff, phen])
        all_data.append([x_exp, y_exp_norm, y, phen])
    else:
        continue

Сохранить параметры модели для каждого фенотипа в csv

In [22]:
dd = []
for i in range(len(all_data)):
    ss = []
    for j in range(3):
        ss.append(parametrs[i][0][j])
    dd.append([ss + [str(parametrs[i][1])]])

data = [tuple(i[0]) for i in dd]
labels = ['pp', 'pn', 'pe', 'phenotype']
par_df = pd.DataFrame.from_records(parametrs)
par_df.to_csv("Phenotypes_params.csv")

Визуализация. Красная линия - экспериментальное распределение, синяя - смоделированное. 

In [9]:
fig = plt.figure(figsize=(25, 25)) #visualization
columns = 4
rows = 4
for i in range(1, 10):
    fig.add_subplot(rows, columns, i)
    plt.plot(all_data[i - 1][0], all_data[i - 1][1], color="red")
    plt.plot(all_data[i - 1][0], all_data[i - 1][2], color="blue")
    plt.xlabel("Length of N1")
    plt.ylabel("Prob")
    plt.title(str(all_data[i - 1][3])[2:30])
plt.savefig('pheno.png', bbox_inches='tight')

Попытка проверить модель с помощью хи-квадрат с треском провалилась. Нужны не частоты, а число наблюдений?

In [82]:
for i in range(len(parametrs)):
    print(chisquare(all_data[i][1],  f_exp=all_data[i][2], ddof = 3), str(all_data[i][3])[2:30] )
    

Power_divergenceResult(statistic=0.18998743757861111, pvalue=1.0) Acute viral infection
Power_divergenceResult(statistic=0.13614530441209263, pvalue=1.0) Chronic lymphocytic leukemia
Power_divergenceResult(statistic=0.087993912993141132, pvalue=1.0) Health bone marrow
Power_divergenceResult(statistic=0.03867469328035239, pvalue=1.0) Health periphery
Power_divergenceResult(statistic=0.31381790883448235, pvalue=1.0) Healthy elderly humans
Power_divergenceResult(statistic=0.16819723585696292, pvalue=1.0) Hepatitis C virus infection
Power_divergenceResult(statistic=0.61648695904553352, pvalue=1.0) Hodgkin lymphoma
Power_divergenceResult(statistic=0.24775715281716892, pvalue=1.0) Infectious mononucleosis
Power_divergenceResult(statistic=1.1064663613593411, pvalue=1.0) Multiple sclerosis
Power_divergenceResult(statistic=0.79413202914435765, pvalue=1.0) Non-Hodgkin lymphomas
Power_divergenceResult(statistic=0.4306546107603445, pvalue=1.0) Ocular adnexal lymphoma
Power_divergenceResult(statist

In [85]:
for i in range(len(parametrs)):
    print(chisquare([i*1000 for i in all_data[i][1]],  f_exp=[i*1000 for i in all_data[i][2]], ddof = 3), str(all_data[i][3])[2:30] )
    

Power_divergenceResult(statistic=189.98743757861112, pvalue=2.069917417123229e-19) Acute viral infection
Power_divergenceResult(statistic=136.14530441209266, pvalue=7.5225230148338552e-11) Chronic lymphocytic leukemia
Power_divergenceResult(statistic=87.993912993141123, pvalue=0.00019109394535218936) Health bone marrow
Power_divergenceResult(statistic=38.674693280352386, pvalue=0.76972869342499217) Health periphery
Power_divergenceResult(statistic=313.81790883448235, pvalue=1.4923870546006443e-41) Healthy elderly humans
Power_divergenceResult(statistic=168.19723585696292, pvalue=7.9498494019800591e-16) Hepatitis C virus infection
Power_divergenceResult(statistic=616.48695904553324, pvalue=7.387534169085388e-101) Hodgkin lymphoma
Power_divergenceResult(statistic=247.75715281716896, pvalue=1.9027540179074406e-29) Infectious mononucleosis
Power_divergenceResult(statistic=1106.4663613593411, pvalue=1.1083730661465275e-201) Multiple sclerosis
Power_divergenceResult(statistic=794.13202914435

In [16]:
all_data[3][1]

[0.069767441860465115,
 0.071881606765327691,
 0.10782241014799154,
 0.11416490486257928,
 0.084566596194503171,
 0.076109936575052856,
 0.063424947145877375,
 0.07399577167019028,
 0.057082452431289642,
 0.040169133192389003,
 0.044397463002114168,
 0.046511627906976744,
 0.040169133192389003,
 0.029598308668076109,
 0.023255813953488372,
 0.014799154334038054,
 0.0063424947145877377,
 0.0084566596194503175,
 0.0042283298097251587,
 0.012684989429175475,
 0.0042283298097251587,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0042283298097251587,
 0.0,
 0.0,
 0.0021141649048625794,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [12]:
parametrs[3]

[array([ 0.2       ,  0.07460483,  0.30957584]), '//Health bone marrow']

In [15]:
list(phen_hist_dict["//Health bone marrow"][:50])

[33.0,
 34.0,
 51.0,
 54.0,
 40.0,
 36.0,
 30.0,
 35.0,
 27.0,
 19.0,
 21.0,
 22.0,
 19.0,
 14.0,
 11.0,
 7.0,
 3.0,
 4.0,
 2.0,
 6.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]